In [1]:
import pandas as pd
import geopandas as gpd
from config import *

In [2]:
csvs = {
    2010: pd.read_csv('combined2010.csv'),
    2015: pd.read_csv('combined2015.csv'),
    2019: pd.read_csv('combined2019.csv'),
    2020: pd.read_csv('combined2020.csv'),
    2021: pd.read_csv('combined2021.csv')
}
codebook = pd.read_excel('Codebook.xlsx')

In [3]:
mig = gpd.GeoDataFrame(pd.read_pickle(dataFolder + '2010-2022_net_migration.pkl'))

mig = mig.reset_index().rename(columns={
    'cbsa': 'CBSA',
    'NPOPCHG_2010': 'NPOPCHG2010',
    'NPOPCHG_2011': 'NPOPCHG2011',
    'NPOPCHG_2012': 'NPOPCHG2012',
    'NPOPCHG_2013': 'NPOPCHG2013',
    'NPOPCHG_2014': 'NPOPCHG2014',
    'NPOPCHG_2015': 'NPOPCHG2015',
    'NPOPCHG_2016': 'NPOPCHG2016',
    'NPOPCHG_2017': 'NPOPCHG2017',
    'NPOPCHG_2018': 'NPOPCHG2018',
    'NPOPCHG_2019': 'NPOPCHG2019',
    'NPOPCHG_2020': 'NPOPCHG2020',
    'NATURALCHG2021': 'NATURALINC2021',
    'RNATURALCHG2021': 'RNATURALINC2021',
    'NATURALCHG2022': 'NATURALINC2022',
    'RNATURALCHG2022': 'RNATURALINC2022'
}).drop(columns=[
    'STNAME', 'CTYNAME', 'CENSUS2010POP' #CENSUS2010POP is a string??
])

In [4]:
data = pd.DataFrame(index=pd.Index([], name='CBSA'))

for year, csv in csvs.items():
    csv.set_index('CBSA', inplace=True)
    csv.rename(columns={
        name: (name, year) for name in csv.columns
    }, inplace=True)

    data = pd.merge(data, csv, on='CBSA', how='outer')

In [5]:
migCols = [
    'POPESTIMATE',
    'NPOPCHG',
    'BIRTHS',
    'DEATHS',
    'NATURALINC',
    'INTERNATIONALMIG',
    'DOMESTICMIG',
    'NETMIG',
    'RESIDUAL',
    'GQESTIMATES'
]

migRCols = [
    'RBIRTH',
    'RDEATH',
    'RNATURALINC',
    'RINTERNATIONALMIG',
    'RDOMESTICMIG',
    'RNETMIG'
]

newNames = {}

for col in migCols:
    for year in range(2010, 2022+1):
        newNames[col + str(year)] = (col, year)

for col in migRCols:
    for year in range(2011, 2022+1):
        newNames[col + str(year)] = (col, year)

mig = mig.set_index('CBSA')[newNames.keys()].rename(columns=newNames)

In [6]:
data = data.drop(columns=[c for c in mig.columns if c in data.columns])
data = data.drop(columns=[('NPOPCHG_', 2010), ('NPOPCHG_', 2015), ('NPOPCHG_', 2019)])
data = pd.merge(data, mig, on='CBSA', how='outer')

In [7]:
cols = []

for i, row in codebook.iterrows():
    name = row['column name']
    years = str(row['year(s)'])
    
    if name in {'CBSA', 'geometry'}: continue
    
    for year in years.split(','):
        year = int(year)
        cols.append((name, year))

for col in sorted(data.columns):
    if not col in cols:
        print(col, "was removed")

data = data[cols]

('LSAD', 2010) was removed
('LSAD', 2015) was removed
('LSAD', 2019) was removed
('LSAD', 2021) was removed
('NAME', 2010) was removed
('NAME', 2015) was removed
('NAME', 2020) was removed
('NAME', 2021) was removed
('Num_Comp_Info_Res', 2010) was removed
('Num_Comp_Info_Res', 2015) was removed
('Num_Comp_Info_Res', 2019) was removed
('Num_Comp_Info_Res', 2021) was removed
('Num_Comp_Programmer', 2010) was removed
('Num_Comp_Programmer', 2015) was removed
('Num_Comp_Programmer', 2019) was removed
('Num_Comp_Programmer', 2021) was removed
('Num_Comp_Sys_Analyst', 2010) was removed
('Num_Comp_Sys_Analyst', 2015) was removed
('Num_Comp_Sys_Analyst', 2019) was removed
('Num_Comp_Sys_Analyst', 2021) was removed
('Num_Info_Sec_Analyst', 2010) was removed
('Num_Info_Sec_Analyst', 2015) was removed
('Num_Info_Sec_Analyst', 2019) was removed
('Num_Info_Sec_Analyst', 2021) was removed
('Num_Soft_Dev', 2010) was removed
('Num_Soft_Dev', 2015) was removed
('Num_Soft_Dev', 2019) was removed
('Num_S

In [8]:
data.columns = pd.MultiIndex.from_tuples(data.columns, names=('Variable', 'Year')) #Create MultiIndex
data

Variable                                  NAME                           LSAD  \
Year                                      2019                           2020   
CBSA                                                                            
12020                 Athens-Clarke County, GA  Metropolitan Statistical Area   
12060     Atlanta-Sandy Springs-Alpharetta, GA  Metropolitan Statistical Area   
12100              Atlantic City-Hammonton, NJ  Metropolitan Statistical Area   
12120                               Atmore, AL  Micropolitan Statistical Area   
12140                               Auburn, IN  Micropolitan Statistical Area   
...                                        ...                            ...   
49080                           Winnemucca, NV  Micropolitan Statistical Area   
49100                               Winona, MN  Micropolitan Statistical Area   
49180                        Winston-Salem, NC  Metropolitan Statistical Area   
49220          Wisconsin Rapids-Marshfield, WI  Micropolitan Statistical Area   
14160                                      NaN                            NaN   

Variable  metro  micro inBuff commuteBuffCount            workingPop  \
Year       2021   2021   2019             2019       2020       2019   
CBSA                                                                   
12020     False  False   True          21008.0    20168.0    80835.0   
12060      True  False   True        2412279.0  2346403.0  2590292.0   
12100     False  False   True          27041.0    24765.0   123380.0   
12120     False   True  False              0.0        0.0    12416.0   
12140     False   True   True            728.0      624.0    20907.0   
...         ...    ...    ...              ...        ...        ...   
49080     False   True  False              0.0        0.0     7520.0   
49100     False   True   True           1488.0     1382.0    22024.0   
49180     False  False   True          41421.0    40087.0   290914.0   
49220     False   True  False              0.0        0.0    37117.0   
14160       NaN    NaN    NaN              NaN        NaN        NaN   

Variable            commutePercent  ... Tot_Bldgs          Tot_Bldgs_Value  \
Year           2020           2019  ...      2019     2021            2010   
CBSA                                ...                                      
12020       78919.0      25.988743  ...     947.0    928.0    4.513729e+07   
12060     2517685.0      93.127686  ...   26484.0  31960.0    1.284808e+09   
12100      106316.0      21.916842  ...     421.0    575.0    7.940623e+07   
12120       11212.0       0.000000  ...       7.0     10.0    4.594400e+05   
12140       20137.0       3.482087  ...     150.0    138.0    1.119984e+07   
...             ...            ...  ...       ...      ...             ...   
49080        7393.0       0.000000  ...      14.0     31.0    1.831417e+06   
49100       20277.0       6.756266  ...      57.0     62.0    1.520123e+07   
49180      280860.0      14.238228  ...    3176.0   3911.0    1.914697e+08   
49220       35449.0       0.000000  ...     114.0    117.0    1.372929e+07   
14160           NaN            NaN  ...       NaN      NaN             NaN   

Variable                                           Tot_Units           \
Year              2015          2019          2021      2010     2015   
CBSA                                                                    
12020     1.834011e+08  2.605810e+08  3.753031e+08     226.0   1159.0   
12060     5.318834e+09  6.783502e+09  8.721861e+09    7608.0  30342.0   
12100     1.122180e+08  1.477606e+08  1.940337e+08     512.0    710.0   
12120     1.740280e+05  9.137180e+05  1.445752e+06       7.0      2.0   
12140     1.933319e+07  4.038081e+07  4.435193e+07      70.0     98.0   
...                ...           ...           ...       ...      ...   
49080     6.324198e+06  2.396881e+06  5.397395e+06      18.0     51.0   
49100     1.223176e+07  1.457054e

In [9]:
data.to_csv('dataset.csv')
data.to_excel('dataset.xlsx')

In [10]:
df = pd.DataFrame()

print('CBSA', len(data.index))

for name in data.columns:
    name = tuple(name)
    col = data[name]
    print(name, (~col.isna()).sum())
    # df.loc['missing', name] = float(col.isna().sum())
    # df.loc['missing %', name] = col.isna().mean() * 100
    # df.loc['zeros', name] = (col == 0).sum()
    # df.loc['zeros %', name] = (col == 0).mean() * 100

# df = df.T
# df

CBSA 927
('NAME', 2019) 926
('LSAD', 2020) 926
('metro', 2021) 926
('micro', 2021) 926
('inBuff', 2019) 926
('commuteBuffCount', 2019) 926
('commuteBuffCount', 2020) 926
('workingPop', 2019) 926
('workingPop', 2020) 926
('commutePercent', 2019) 926
('commutePercent', 2020) 926
('liminal', 2020) 926
('Days with AQI', 2010) 519
('Days with AQI', 2015) 512
('Days with AQI', 2019) 502
('Days with AQI', 2021) 493
('Good Days', 2010) 519
('Good Days', 2015) 512
('Good Days', 2019) 502
('Good Days', 2021) 493
('Moderate Days', 2010) 519
('Moderate Days', 2015) 512
('Moderate Days', 2019) 502
('Moderate Days', 2021) 493
('Unhealthy for Sensitive Groups Days', 2010) 519
('Unhealthy for Sensitive Groups Days', 2015) 512
('Unhealthy for Sensitive Groups Days', 2019) 502
('Unhealthy for Sensitive Groups Days', 2021) 493
('Unhealthy Days', 2010) 519
('Unhealthy Days', 2015) 512
('Unhealthy Days', 2019) 502
('Unhealthy Days', 2021) 493
('Very Unhealthy Days', 2010) 519
('Very Unhealthy Days', 2015) 